In [1]:
#!pip install optuna optuna-dashboard
#!pip install jupyterlab_autoscrollcelloutput

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 638 kB 10.0 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
import utils
from utils import logger

In [ ]:
import os
import sys
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import engine_v3
import utils
import time
import optuna
import torch.optim as optim

from optuna.trial import TrialState
from logger import OutputLogger
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization, training, extract_face
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import distributed
from losses import CombinedMarginLoss, ArcFace
from metrics import ArcMarginProduct
#from torch.optim.lr_scheduler import StepLR, PolynomialLR

In [ ]:
def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int
):
    """
    Creates training and testing DataLoaders.
    Takes in a training directory and testing directory path and turns
    them into PyTorch Datasets and then into PyTorch DataLoaders.
    Args:
      train_dir: Path to training directory.
      test_dir: Path to testing directory.
      transform: torchvision transforms to perform on training and testing data.
      batch_size: Number of samples per batch in each of the DataLoaders.
      num_workers: An integer for number of workers per DataLoader.
    Returns:
      A tuple of (train_dataloader, test_dataloader, class_names).
      Where class_names is a list of the target classes.
      Example usage:
        train_dataloader, test_dataloader, class_names = \
          = create_dataloaders(train_dir=path/to/train_dir,
                               test_dir=path/to/test_dir,
                               transform=some_transform,
                               batch_size=32,
                               num_workers=4)
    """
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)
    img,label = train_data[0][0], train_data[0][1]

    # Get class names
    class_names = train_data.classes
    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names

In [ ]:
def model_summary(model): 
    
    try:
        import torchinfo
    except ModuleNotFoundError:
        !pip install torchinfo
        import torchinfo
        
    summary = torchinfo.summary(model=model, 
        input_size=(32, 3, 160, 160), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
    )
    
    return summary

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'

In [ ]:
epochs = 3
batch_size = 128
num_classes = 10000
#sample_rate = 1.0
#momentum = 0.9
#weight_decay = 5e-4
num_workers = os.cpu_count()

In [ ]:
#criterion = torch.nn.CrossEntropyLoss()
#metric_fc = ArcMarginProduct(512, num_classes, s=30, m=0.5, easy_margin=False)
##optimizer = torch.optim.SGD([{'params': facenet.parameters()}, {'params': metric_fc.parameters()}, {'params': criterion.parameters()}], momentum = momentum, lr = learning_rate, weight_decay = weight_decay)
#optimizer = torch.optim.Adam([{'params': facenet.parameters()}, {'params': metric_fc.parameters()}, {'params': criterion.parameters()}], lr = learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay = weight_decay)
#scheduler = PolynomialLR(optimizer, epochs, 2)
##scheduler = StepLR(optimizer, step_size=2, gamma=0.01)
#
## Create a log file name
#log_filename = "output.log"


In [ ]:
def objective(trial):
    print("HERE")
    model = InceptionResnetV1(classify=False, pretrained='casia-webface').to(device)
    for param in model.parameters():
        param.requires_grad = False # Freezes all the layers
        
    s = trial.suggest_int("scale", 6, 64, 2)
    m = trial.suggest_float("margin", 1e-4, 5e-1, log=True)
        
    criterion = torch.nn.CrossEntropyLoss()
    metric_fc = ArcMarginProduct(512, num_classes, s=s, m=m, easy_margin=False)
    #
    #unfreeze_layers = [
    #                   facenet.mixed_7a,
    #                   facenet.repeat_3, 
    #                   facenet.block8, 
    #                   facenet.avgpool_1a, 
    #                   facenet.dropout, 
    #                  facenet.last_linear, 
    #                    facenet.last_bn]
    
    #for layer in unfreeze_layers:
    #    for param in layer.parameters():
    #        param.requires_grad = True
        
    # Generate the optimizers.
    #optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    #optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    optimizer = torch.optim.Adam([{'params': model.parameters()}, {'params': metric_fc.parameters()}, {'params': criterion.parameters()}], lr = lr)

    train_dir = "/test_cuda/digiface_cropped/train_dir"
    test_dir = "/test_cuda/digiface_cropped/test_dir"
    data_transforms = transforms.Compose([
                                        np.float32,
                                        transforms.ToTensor(),
                                        fixed_image_standardization
                                        ])
    
    train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir, test_dir, data_transforms, batch_size, num_workers)
    
    #num_classes = len(class_names)
    print(lr, s, m)
    lfw, xqlfw, cplfw = engine_v3.train(model = model, 
             train_dataloader = train_dataloader, 
             test_dataloader = test_dataloader,
             metric_fc = metric_fc,
             #scheduler = scheduler,
             criterion = criterion,
             optimizer = optimizer,
             epochs = epochs,
             device = device)

    return lfw, xqlfw, cplfw

In [ ]:
#
#log_filename = "optuna.txt"
#with open(log_filename, "a") as log_file:
## Create an instance of the OutputLogger
#output_logger = OutputLogger(log_file)
## Set sys.stdout to the OutputLogger instance
#sys.stdout = output_logger

study = optuna.create_study(directions=['maximize', 'maximize', 'maximize'], study_name = 'Facenet + ArcFace hyperparameter tuning')
study.optimize(objective, n_trials=30, timeout=600, show_progress_bar = True)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))
#print("Best trial:")
#trial = study.best_trial
#print("  Value: ", trial.value)
#print("  Params: ")
#for key, value in trial.params.items():
    #print("    {}: {}".format(key, value))
    
#sys.stdout = sys.__stdout__
#log_file.close()